In [1]:
import numpy as np
import tensorflow as tf

In [2]:
# CUSTOMIZABLE: Collect/Prepare data
datapoint_size = 1000
batch_size = 5
steps = 10000
actual_W = 2
actual_b = 10
learn_rate = 0.001
log_file = "/tmp/feature_1_batch_1"

In [3]:
# Model linear regression y = Wx + b
x = tf.placeholder(tf.float32, [None, 1], name="x")
W = tf.Variable(tf.zeros([1,1]), name="W")
b = tf.Variable(tf.zeros([1]), name="b")

print(x)
print(W)
print(b)

Tensor("x:0", shape=(?, 1), dtype=float32)
<tf.Variable 'W:0' shape=(1, 1) dtype=float32_ref>
<tf.Variable 'b:0' shape=(1,) dtype=float32_ref>


In [4]:
with tf.name_scope("Wx_b") as scope:
    product = tf.matmul(x,W)
    y = product + b
    
print(product)
print(y)

Tensor("Wx_b/MatMul:0", shape=(?, 1), dtype=float32)
Tensor("Wx_b/add:0", shape=(?, 1), dtype=float32)


In [5]:
# Add summary ops to collect data
W_hist = tf.summary.histogram("weights", W)
b_hist = tf.summary.histogram("biases", b)
y_hist = tf.summary.histogram("y", y)

print(W_hist)
print(b_hist)
print(y_hist)

Tensor("weights:0", shape=(), dtype=string)
Tensor("biases:0", shape=(), dtype=string)
Tensor("y:0", shape=(), dtype=string)


In [6]:
y_ = tf.placeholder(tf.float32, [None, 1], name="y_")
print(y_)

Tensor("y_:0", shape=(?, 1), dtype=float32)


In [7]:
# Cost function sum((y_-y)**2)
with tf.name_scope("cost") as scope:
    cost = tf.reduce_mean(tf.square(y_-y))
    cost_sum = tf.summary.scalar("cost", cost)
    
print(cost)
print(cost_sum)

Tensor("cost/Mean:0", shape=(), dtype=float32)
Tensor("cost/cost:0", shape=(), dtype=string)


In [8]:
# Training using Gradient Descent to minimize cost
with tf.name_scope("train") as scope:
    train_step = tf.train.GradientDescentOptimizer(learn_rate).minimize(cost)

print(train_step)

name: "train/GradientDescent"
op: "NoOp"
input: "^train/GradientDescent/update_W/ApplyGradientDescent"
input: "^train/GradientDescent/update_b/ApplyGradientDescent"



In [9]:
all_xs = []
all_ys = []
for i in range(datapoint_size):
    # Create fake data for y = W.x + b where W = 2, b = actual_b
    all_xs.append(i%10)
    all_ys.append(actual_W*(i%10)+actual_b)
    
print(type(all_xs))
print(type(all_ys))

<class 'list'>
<class 'list'>


In [10]:
all_xs = np.transpose([all_xs])
all_ys = np.transpose([all_ys])

print(all_xs.shape)
print(all_ys.shape)

(1000, 1)
(1000, 1)


In [11]:
sess = tf.Session()

init = tf.initialize_all_variables()
sess.run(init)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [12]:
# Merge all the summaries and write them out to /tmp/mnist_logs
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter(log_file, sess.graph_def)

In [17]:
print(datapoint_size)
print(batch_size)

1000
5


In [21]:
steps = 20

In [22]:
for i in range(steps):
    if datapoint_size == batch_size:
        batch_start_idx = 0
    elif datapoint_size < batch_size:
        raise ValueError("datapoint_size: %d, must be greater than batch_size: %d" % (datapoint_size, batch_size))
    else:
        batch_start_idx = (i * batch_size) % (datapoint_size - batch_size)
        
    batch_end_idx = batch_start_idx + batch_size
    batch_xs = all_xs[batch_start_idx:batch_end_idx]
    batch_ys = all_ys[batch_start_idx:batch_end_idx]
    
    xs = np.array(batch_xs)
    ys = np.array(batch_ys)
    
    print(xs)
    print(ys)
    
    # Record summary data, and the accuracy every 10 steps
    if i % 10 == 0:
        all_feed = { x: all_xs, y_: all_ys }
        result = sess.run(merged, feed_dict=all_feed)
        writer.add_summary(result, i)
    else:
        feed = { x: xs, y_: ys }
        sess.run(train_step, feed_dict=feed)
        print("y: %s" % sess.run(y, feed_dict=feed))
        print("y_: %s" % ys)
        print("cost: %f" % sess.run(cost, feed_dict=feed))
    
    print("After %d iteration:" % i)
    print("W: %f" % sess.run(W))
    print("b: %f" % sess.run(b))

# NOTE: W should be close to actual_W, and b should be close to actual_b

[[0]
 [1]
 [2]
 [3]
 [4]]
[[10]
 [12]
 [14]
 [16]
 [18]]
After 0 iteration:
W: 0.760529
b: 0.141672
[[5]
 [6]
 [7]
 [8]
 [9]]
[[20]
 [22]
 [24]
 [26]
 [28]]
y: [[ 5.30359888]
 [ 6.32857037]
 [ 7.35354185]
 [ 8.37851334]
 [ 9.40348434]]
y_: [[20]
 [22]
 [24]
 [26]
 [28]]
cost: 279.005920
After 1 iteration:
W: 1.024971
b: 0.178741
[[0]
 [1]
 [2]
 [3]
 [4]]
[[10]
 [12]
 [14]
 [16]
 [18]]
y: [[ 0.20228402]
 [ 1.2782408 ]
 [ 2.35419774]
 [ 3.43015456]
 [ 4.50611115]]
y_: [[10]
 [12]
 [14]
 [16]
 [18]]
cost: 137.332428
After 2 iteration:
W: 1.075957
b: 0.202284
[[5]
 [6]
 [7]
 [8]
 [9]]
[[20]
 [22]
 [24]
 [26]
 [28]]
y: [[  6.77170229]
 [  8.07907867]
 [  9.38645649]
 [ 10.69383335]
 [ 12.00121021]]
y_: [[20]
 [22]
 [24]
 [26]
 [28]]
cost: 214.515137
After 3 iteration:
W: 1.307377
b: 0.234816
[[0]
 [1]
 [2]
 [3]
 [4]]
[[10]
 [12]
 [14]
 [16]
 [18]]
y: [[ 0.25711691]
 [ 1.61186635]
 [ 2.96661568]
 [ 4.32136488]
 [ 5.67611456]]
y_: [[10]
 [12]
 [14]
 [16]
 [18]]
cost: 122.568268
After 4 iterat